In [2]:
# !pip install imutils
# !pip install tqdm
# !conda install keras -y



import os
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import shutil
from IPython.display import clear_output
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from brain_tumor.preprocess import load_data, crop_imgs, save_new_images, preprocess_imgs

clear_output()

role = get_execution_role()

bucket = 'neuroati-machine-learning'# enter your s3 bucket where you will copy data and model artifacts
prefix = 'brain-tumor/dataset'
dataset_file = 'brain-mri-images-for-brain-tumor-detection.zip'

s3 = boto3.client('s3')
!rm -rf 'dataset' TRAIN TEST VAL TRAIN_CROP TEST_CROP VAL_CROP keras-pretrained
!mkdir 'dataset'
s3.download_file(bucket, os.path.join(prefix, dataset_file), os.path.join('dataset', dataset_file))

!unzip 'dataset/brain-mri-images-for-brain-tumor-detection.zip' -d 'dataset/'
!rm 'dataset/brain-mri-images-for-brain-tumor-detection.zip'

!mkdir TRAIN TEST VAL TRAIN/YES TRAIN/NO TEST/YES TEST/NO VAL/YES VAL/NO

IMG_PATH = './dataset/brain_tumor_dataset/'
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < 5:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)

                

TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224,224)

# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

X_train_crop = crop_imgs(set_name=X_train)
X_val_crop = crop_imgs(set_name=X_val)
X_test_crop = crop_imgs(set_name=X_test)


Archive:  dataset/brain-mri-images-for-brain-tumor-detection.zip
   creating: dataset/brain_tumor_dataset/
   creating: dataset/brain_tumor_dataset/no/
  inflating: dataset/brain_tumor_dataset/no/30 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/22 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/41 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/14 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/no 10.jpg  
  inflating: dataset/brain_tumor_dataset/no/18 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/no 9.png  
  inflating: dataset/brain_tumor_dataset/no/no 7.jpeg  
  inflating: dataset/brain_tumor_dataset/no/no 8.jpg  
  inflating: dataset/brain_tumor_dataset/no/no.jpg  
  inflating: dataset/brain_tumor_dataset/no/3 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/43 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/20 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/32 no.jpg  
  inflating: dataset/brain_tumor_dataset/no/2 no.jpeg  
  inflating: d

  inflating: dataset/brain_tumor_dataset/yes/Y2.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y147.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y153.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y108.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y120.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y242.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y256.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y18.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y24.jpg  
 extracting: dataset/brain_tumor_dataset/yes/Y30.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y31.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y25.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y19.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y257.jpg  
  inflating: dataset/brain_tumor_dataset/yes/Y243.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y109.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y255.JPG  
  inflating: dataset/brain_tumor_dataset/yes/Y33.jpg  
 

100%|██████████| 2/2 [00:00<00:00, 12.76it/s]

193 images loaded from TRAIN/ directory.
10 images loaded from TEST/ directory.
50 images loaded from VAL/ directory.


In [3]:




# saving new images to the folder
!rm -rf TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO
!mkdir TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO

save_new_images(X_train_crop, y_train, folder_name='TRAIN_CROP/')
save_new_images(X_val_crop, y_val, folder_name='VAL_CROP/')
save_new_images(X_test_crop, y_test, folder_name='TEST_CROP/')
